In [1]:
import anywidget
import traitlets
from traitlets import HasTraits, observe

class SyncWidget(anywidget.AnyWidget, HasTraits):
    _esm = """
    function render({ model, el }) {
			model.on("msg:custom", (msg) => {
                console.log(msg);
    			if (msg.type == "msg:sync_values"){
    				console.log('syncing values');
    				const values = model.get('values');
    				model.set('values', values.map(val => val + 5));
    				model.save_changes();
                    setTimeout(() => { model.send({ type: "sync_complete" }); }, 100);
    			}
                console.log("synchronizing");
            });
    };
    export default { render };
    """
    values = traitlets.List([]).tag(sync=True)

    @observe("values")
    def _observe_values(self, change):
        print(f"Old: {change['old']}")
        print(f"New: {change['new']}")
    
    def on_msg(self, msg):
        if msg["type"] == "sync_complete":
            print(f"Received sync_complete: {msg['values']}")

In [2]:
widget = SyncWidget()
widget

SyncWidget()

In [3]:
widget.values = [1,2,3,4,5]

Old: []
New: [1, 2, 3, 4, 5]


In [4]:
widget.values

[1, 2, 3, 4, 5]

In [5]:
widget.send({ "type" : "msg:sync_values"})
print(widget.values)

[1, 2, 3, 4, 5]


In [6]:
widget.values

[1, 2, 3, 4, 5]

In [7]:
# dir(widget)

In [8]:
# widget.get_state()['values']
# widget.traits()['values'].get('values')
widget.values = widget.values

In [9]:
widget.values

[1, 2, 3, 4, 5]

In [10]:
from IPython import get_ipython
get_ipython().kernel.do_one_iteration()
print(widget.values)

[1, 2, 3, 4, 5]


/var/folders/0l/b3dbb2_d2bb4y3wbbfk0wt_80000gn/T/ipykernel_20798/1794953676.py:2: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  get_ipython().kernel.do_one_iteration()


In [11]:
widget.values

[1, 2, 3, 4, 5]